<a href="https://colab.research.google.com/github/adrirens/MML_course/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'MyDrive/mml'       # path to your project on Drive

drive.mount(ROOT)           # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)
!mkdir "{PROJECT_PATH}"    # in case we haven't created it already
%cd "{PROJECT_PATH}"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/MyDrive/mml’: File exists
/content/drive/MyDrive/mml


In [3]:
GIT_USERNAME = "adrirens"  # This is a shared repository. If you want to synchronize with your own fork, well, you need to fork the github repository and replace the given name by your username.
GIT_TOKEN = "XXX"  # This token is used only if you work with your own fork. You have to generate it and put it here. Make sure to keep it confidential: it is a very sensitive information!
GIT_REPOSITORY = "MML_course"

In [4]:
# GIT_PATH = "https://"+GIT_TOKEN+"@github.com/"+GIT_USERNAME+"/"+GIT_REPOSITORY+".git"
#GIT_PATH = "https://github.com/"+GIT_USERNAME+"/"+GIT_REPOSITORY+".git"
#!git clone "{GIT_PATH}"
%cd "{GIT_REPOSITORY}"

/content/drive/MyDrive/mml/MML_course


In [5]:
!ls

base_model  figures		  image_classification_conv  model
code.ipynb  function_fitting	  logs			     README.md
dataset     image_classification  MMl_course		     solving_pde


In [6]:
!pip install torch torchvision fvcore iopath yacs tqdm numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61397 sha256=79b23ccd642487175f186cef4f55fdb6f44957fcba51a9d3b8a55d7e578b5812
  Stored in directory: /root/.cache/pip/wheels/ed/9f/a5/e4f5b27454ccd4596bd8b62432c7d6b1ca9fa22aef9d70a16a
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=5946148259f0b916c13ef85e1fdfbc9bb9cb736156347aef646cda9613740732
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built fvcore iopath


In [ ]:
models = [
    "I.6.20a",
    "I.6.20",
    "I.6.20b",
    "I.8.4",
    "I.9.18",
    "I.12.2",
    "I.12.4",
    "I.12.5",
    "I.10.7",
    "I.12.11",
    "I.13.4",
    "I.13.12",
    "I.14.3",
    "I.14.4",
    "I.15.3x",
    "I.15.10",
    "I.18.4"
]

grid_sizes = [
    "5",
    "20"
]


model_types = ["KAN", "HyperKAN"]



!python image_classification_conv/train_meta.py \
  --model MetaKAN8_M \
  --n_metanets 1 \
  --optim_set double \
  --lr_h 1e-4 \
  --lr_e 1e-3 \
  --embedding_dim 1 \
  --hidden_dim 32 \
  --dataset CIFAR10 \
  --batch-size 128 \
  --epochs 50
!python image_classification_conv/train.py \
  --model KAN8 \
  --grid_size 5 \
  --spline_order 3 \
  --dataset CIFAR10 \
  --batch-size 128 \
  --epochs 50 \
  --lr 1e-3

In [11]:
import subprocess
import re
import pandas as pd
import time

def run_and_extract(cmd):
    """Exécute une commande shell et retourne les métriques extraites"""
    result = subprocess.run(cmd, capture_output=True, text=True)
    out = result.stdout

    # Extraction des infos
    num_params = re.search(r"Number of parameters:\s*([\d,]+)", out)
    test_losses = re.findall(r"test set: Average loss:\s*([\d\.Ee-]+)", out)
    train_losses = re.findall(r"train set: Average loss:\s*([\d\.Ee-]+)", out)
    time_total = re.search(r"total training time:\s*([\d\.Ee-]+)", out)
    avg_time = re.search(r"average training time per epoch:\s*([\d\.Ee-]+)", out)

    return {
        "num_params": int(num_params.group(1).replace(",", "")) if num_params else None,
        "final_train_loss": float(train_losses[-1]) if train_losses else None,
        "final_test_loss": float(test_losses[-1]) if test_losses else None,
        "total_time_sec": float(time_total.group(1)) if time_total else None,
        "avg_epoch_time": float(avg_time.group(1)) if avg_time else None,
    }

raw_results = []
for grid in grid_sizes
  for model_name in models:
    print(f"\n🔹 Dataset: {dataset} | Grid: {grid}")
    for model_type in model_types:
        print(f"   → Training {model_type}...")

        if model_type == "KAN":
            cmd = [
                "python", "function_fitting/train.py",
                "--model", "KAN",
                "--optimizer", "lbfgs",
                "--lr", "1",
                "--dataset", dataset,
                "--layers_width", "5", "5", "5",
                "--loss", "mse",
                "--kan_bspline_grid", str(grid)
            ]
        else:  # HyperKAN
            cmd = [
                "python", "function_fitting/train_hyper.py",
                "--model", "HyperKAN",
                "--optimizer", "lbfgs",
                "--lr", "1",
                "--dataset", dataset,
                "--layers_width", "5", "5", "5",
                "--loss", "mse",
                "--embedding_dim", "1",
                "--hidden_dim", "8",
                "--kan_bspline_grid", str(grid)
            ]

        res = run_and_extract(cmd)
        res.update({
            "dataset": dataset,
            "grid": grid,
            "type": model_type
        })
        raw_results.append(res)
        time.sleep(0.5)

# 🧾 Conversion en DataFrame
df = pd.DataFrame(raw_results)

# 🪄 Transformation au format du papier
# (colonnes = G=5 KAN, G=5 HyperKAN, G=20 KAN, G=20 HyperKAN)
pivot = df.pivot_table(
    index="dataset",
    columns=["grid", "type"],
    values=["final_test_loss", "num_params"]
)

# Nettoyage du MultiIndex pour affichage plus clair
pivot.columns = [
    f"G={g} {t} {m}" for m, g, t in pivot.columns.to_flat_index()
]

# ✅ Tableau final
pivot = pivot.reset_index()
display(pivot)

# 💾 Sauvegarde CSV
pivot.to_csv("kan_comparison_table.csv", index=False)
print("\n✅ Tableau sauvegardé dans kan_comparison_table.csv")


🔹 Traitement du modèle I.6.20A...

🔹 Traitement du modèle I.6.20...

🔹 Traitement du modèle I.6.20B...

🔹 Traitement du modèle I.8.4...

🔹 Traitement du modèle I.9.18...

🔹 Traitement du modèle I.12.2...

🔹 Traitement du modèle I.12.4...

🔹 Traitement du modèle I.12.5...

🔹 Traitement du modèle I.10.7...

🔹 Traitement du modèle I.12.11...

🔹 Traitement du modèle I.13.4...

🔹 Traitement du modèle I.13.12...

🔹 Traitement du modèle I.14.3...

🔹 Traitement du modèle I.14.4...

🔹 Traitement du modèle I.15.3X...

🔹 Traitement du modèle I.15.10...

🔹 Traitement du modèle I.18.4...


,dataset,type,num_params,final_test_loss,final_train_loss,total_time_sec,avg_epoch_time
0,I.6.20A,KAN,NaN,NaN,NaN,NaN,NaN
1,I.6.20A,HyperKAN,NaN,NaN,NaN,NaN,NaN
2,I.6.20,KAN,1446.0,0.002282,0.002130,56.841894,0.568419
3,I.6.20,HyperKAN,820.0,0.002474,0.002314,96.639826,0.966398
4,I.6.20B,KAN,NaN,NaN,NaN,NaN,NaN
5,I.6.20B,HyperKAN,NaN,NaN,NaN,NaN,NaN
6,I.8.4,KAN,1666.0,0.015662,0.011808,60.292026,0.602920
7,I.8.4,HyperKAN,914.0,0.032943,0.031586,102.164345,1.021643
8,I.9.18,KAN,2216.0,0.001726,0.001565,69.972982,0.699730
9,I.9.18,HyperKAN,1149.0,0.003743,0.003367,111.232485,1.112325



✅ Résultats sauvegardés dans kan_results_summary.csv
